In [2]:
# ==================== #

import nltk
from sklearn.decomposition import TruncatedSVD
import numpy as np
import gensim.downloader as api

START_TOKEN = '<START>'
END_TOKEN = '<END>'

# ==================== #

# Natural Language Processing
---

Das ist eine Zusammenfassung bzw. ein Cheatsheet zum Thema **Natural Language Processing**.<br>
<br>
<br>
<br>
## Table of Contents
<br>

### 1. Basics
* Text Processing
<br>
* Language models
<br>
* Word Vectors
<br>
* Co-Occurence matrix
<br>
* Wortähnlichkeit
<br>

### 2. Spelling Correction
* Noisy Channel Model

<br>

---

<br>

# Basics

## Text Processing
Bevor man mit natürlicher Sprache arbeiten kann, muss man den Text oft erst einmal verarbeiten. Die wichtigsten Schritte:
- **Tokenization:** Zerlegen von Sätzen in einzelne Wörter.
- **Normalization:** Enthält mehrere Schritte, z.
- **Stopword removal:** Entfernen von für eine Sprache typisch häufig vorkommenden und daher nicht wertvollen Wörtern (z.B. 'the').
- **Filtering:** Entfernen von ungewünschten Symbolen, wie z.B. Sonderzeichen. (Meist durch RegEx)
- **Lemmatizing:** Zurückführen aller Wörter auf ihren Wortstamm, d.h. eliminieren jeglicher grammatikalischer Veränderungen.
- **Stemming:** Auch hier Zurückführen aller Wörter auf eine Grundform, aber aggressiver als beim Lemmatizing. Ergebnis ist oft nicht der vollständige, lexikalische Wortstamm.

<br>

In [6]:
# Preprocessing
test_corpus = [
    "Heute ist das Wetter wieder schlecht, denn für den ganzen Tag wurde Regen angesagt.",
    "Präsident Trump hat auch heute wieder unfassbaren Mist auf Twitter geschrieben.",
    "Die Coronavirus-Pandemie hält die Welt weiterhin in Atem: Mehr als 5 Millionen Menschen haben sich weltweit mit dem neuartigen Covid-19 infiziert – 177.183 davon bisher in Deutschland.",
]

# Tokenization
tokenized_sents = [nltk.tokenize.word_tokenize(sentence) for sentence in test_corpus]
print("Tokenized:\n {} \n".format(tokenized_sents))

# Normalization (doesn't need to be done seperately)
for sent in tokenized_sents:
    for idx, word in enumerate(sent):
        sent[idx] = word.lower()
    sent.insert(0, START_TOKEN)
    sent.insert(len(sent), END_TOKEN)
    
print("Normalized:\n {} \n".format(tokenized_sents))

# Join the lists so its easier to process the corpus
tokenized_sents = [token for sentence in tokenized_sents for token in sentence]

# Stopword removal
german_stopwords = nltk.corpus.stopwords.words('german')
sw_removed = [token for token in tokenized_sents if token not in german_stopwords]

print("Length of original: {}".format(len(tokenized_sents)))
print("Length after stopword removal: {} \n".format(len(sw_removed)))

Tokenized:
 [['Heute', 'ist', 'das', 'Wetter', 'wieder', 'schlecht', ',', 'denn', 'für', 'den', 'ganzen', 'Tag', 'wurde', 'Regen', 'angesagt', '.'], ['Präsident', 'Trump', 'hat', 'auch', 'heute', 'wieder', 'unfassbaren', 'Mist', 'auf', 'Twitter', 'geschrieben', '.'], ['Die', 'Coronavirus-Pandemie', 'hält', 'die', 'Welt', 'weiterhin', 'in', 'Atem', ':', 'Mehr', 'als', '5', 'Millionen', 'Menschen', 'haben', 'sich', 'weltweit', 'mit', 'dem', 'neuartigen', 'Covid-19', 'infiziert', '–', '177.183', 'davon', 'bisher', 'in', 'Deutschland', '.']] 

Normalized:
 [['<START>', 'heute', 'ist', 'das', 'wetter', 'wieder', 'schlecht', ',', 'denn', 'für', 'den', 'ganzen', 'tag', 'wurde', 'regen', 'angesagt', '.', '<END>'], ['<START>', 'präsident', 'trump', 'hat', 'auch', 'heute', 'wieder', 'unfassbaren', 'mist', 'auf', 'twitter', 'geschrieben', '.', '<END>'], ['<START>', 'die', 'coronavirus-pandemie', 'hält', 'die', 'welt', 'weiterhin', 'in', 'atem', ':', 'mehr', 'als', '5', 'millionen', 'menschen', 'h

Oft ist es auch nötig, sogenannte N-Gramme zu speichern. Bei diesen handelt es sich einfach nur um Sequenzen von aufeinanderfolgenden
 Wörtern bzw. Charactern im Corpus, wobei N die konkrete Anzahl davon bestimmt. Man unterscheidet zwischen Wort-N-Grammen und Character-N-Grammen.

In [20]:
def token_n_grams(text, n=2):
    """ Creates word-n-grams from a given text
        Params:
            n (int) : The concrete n for the n-grams, so 2 creates Bigrams, 3 creates Trigrams, etc. Default:2
            text (list of strings) : The list of tokens thats used to extract n-grams
            
        Returns:
            token_n_grams (list of strings): List of word-n-grams 
    """
    token_n_grams = [" ".join(text[idx:idx+n]) for idx in range(0, len(text)-n+1)]
    return token_n_grams

def character_n_grams(text, n=2):
    """ Creates character-n-grams from a given text
        Params:
            n (int) : The concrete n for the n-grams, so 2 creates Bigrams, 3 creates Trigrams, etc. Default:2
            text (string) : The text thats used to extract n-grams
            
        Returns:
            char_n_grams (list of strings): List of character-n-grams 
    """
    char_n_grams = [text[idx:idx+2] for idx in range(0, len(text)-n+1)]
    return char_n_grams

sentence = ["Hallo", "das", "ist", "ein", "Test"]

token_bigrams = token_n_grams(sentence)
char_bigrams = character_n_grams(" ".join(sentence))

print("Token Bigrams:\n {} \n".format(token_bigrams))
print("Character Bigrams:\n {} \n".format(char_bigrams))

    
    

Token Bigrams:
 ['Hallo das', 'das ist', 'ist ein', 'ein Test'] 

Character Bigrams:
 ['Ha', 'al', 'll', 'lo', 'o ', ' d', 'da', 'as', 's ', ' i', 'is', 'st', 't ', ' e', 'ei', 'in', 'n ', ' T', 'Te', 'es', 'st'] 



## Language Models
<br>

Language Models können verschiedene NLP Probleme wie Spell Checking, Machine Translation oder auch Speech Recognition lösen. Die Grundidee ist im Endeffekt, für einen bestimmten Satz oder für bestimmte Wörter Wahrscheinlichkeiten auszurechnen.

<br>

#### Spell Checking
Beim Spell Checking möchte man z.B. für einen Satz "The office is about fifteen minuets from my house" erreichen, dass **P("about fifteen minutes from") > P("about fifteen minuets from")** gilt.

#### Machine Translation
Bei der Machine Translation möchte man mit den Wahrscheinlichkeiten gute von schlechten Übersetzungen unterscheiden, z.B. 
**P("High winds tonite") > P("Large winds tonite")**, da erstere eine bessere Übersetzung darstellt.

<br>

Was wir im Endeffekt also möchten ist ein Model, dass für einen String W = w1....wn die Wahrscheinlichkeit
<br>
**<p style="text-align: center;">P(W) = P(w1,w2,....,wn)</p>** oder 

<br>

**<p style="text-align: center;">P(wn | w1,w2,...wn-1)</p>**
<br>
berechnen kann.

<br>

#### Wie werden die Wahrscheinlichkeiten berechnet?
Die Wahrscheinlichkeiten werden folgendermaßen berechnet:

<br>

![Markov Assumption](img/markov-assumption.png)

<br>

Dabei geht es darum, nicht für jedes Wort den gesamten Kontext zu betrachten (zu kompliziert, zu viele Möglichkeiten), sondern man betrachtet für jedes Wort nur k Kontextwörter.
<br>
Für **k=0** nennt man solche Models **Unigram Models**, diese betrachten keinerlei Kontext, d.h. die Wahrscheinlichkeit eines Satzes ist einfach das Produkt der Wahrscheinlichkeiten aller Einzelwörter. Für **k=1** spricht man von **Bigram Models**, die das jeweils hintere Wort betrachten etc. 

<br>

Die Einzelwahrscheinlichkeiten werden über die **Counts** geschätzt. Für ein Bigram Model ginge das folgendermaßen:

<br>

![Count Probability](img/count-probability.png)

<br>

Mit c als "Count-Funktion" berechnet das also, wie oft **wi-1** und **wi** gemeinsam vorkommen geteilt durch die Anzahl der Vorkommen von **wi-1**. Dies wird nun für ein ausreichend großes Dataset für jedes Wort im Trainingsset berechnet. Das Ergebnis ist ein Language Model, dass dann für einen konkretes Problem verwendet werden kann.
<br>
In der Praxis werden diese Wahrscheinlichkeiten aus Effizienzgründen aber nicht miteinander multipliziert, sondern man logarithmiert und addiert sie in der Form **P1 x P2 = log(P1) + log(P2)**.

<br>
    
#### Smoothing
Da es Daten gibt, die im Testset aber nicht im Trainingsset vorkommen, muss zusätzlich Smoothing eingeführt werden, da sonst Wahrscheinlichkeiten schnell den Wert 0 annehmen und so die Performanz negativ beeinflussen. (Außerdem lässt sich nichts durch 0 teilen, was früher oder später dazu führen würde.)
<br>
<br>
Diese Wörter werden durch ein spezielles Token **< UNK >** kodiert, welches in der Regel die Wörter im Trainingsset ersetzt, die sowieso schon sehr selten vorkommen. Dadurch lassen sich dann N-Gram-Wahrscheinlichkeiten für unbekannte Wörter trainieren, welche man dann auf das Testset anwenden kann.

<br>

#### Add-1-Smoothing
Hier addiert man zu den Counts einfach den Wert 1. ( V = Anzahl Wörter, deren count um 1 erhöht wurde )

<br>

![Add-One-Smoothing](img/add-one.jpg)

<br>

Diese Art von Smoothing gilt allerdings als schlecht (zumindest für N-Gram-Models), weshalb bessere Alternativen existieren. (z.B. **Extended Interpolated Kneser Ney** oder für extrem große Datasets: **Stupid Backoff**)

<br>

---

## Word Vectors
<br>

In der Computerlinguistik werden Wörter in der Regel als Vektoren dargestellt, um sie maschinell verarbeiten zu können. 
Dabei können Wörter auf verschiedene Arten als Vektoren dargestellt werden, wobei die konkrete Herangehensweise ziemlich vom Anwendungsfall abhängt. 

<br>

#### One-Hot-Vectors
Bei den sogenannten **One-Hot-Vectors** handelt es sich um Vektoren, die nur aus den Werten 0 und 1 bestehen.
Diese finden meist Anwendung, wenn man gegeben einem **Vokabular V** bestehend aus n verschiedenen Wörtern und 
einer **Dokumentensammlung D** für jedes Dokument d speichern möchte, ob das jeweilige Wort in Dokument d vorkommt oder nicht. In diesem Fall spricht von einer **Term-Document-Matrix**.
<br>
Dabei bestehen die One-Hot-Vectors aus insgesamt n Komponenten (Größe des Vokabulars), wobei der Wert der jeweiligen Vektorkomponente
1 ist, wenn das Wort in Dokument d vorkommt und 0 wenn nicht.
<br>
<br>
"Stapelt" man die One-Hot-Vectors aller Dokumente, ergibt sich eine Matrix mit N Zeilen und K Spalten, wobei N die Anzahl der Dokumente
und K die Vokabulargröße ist. Aus dieser kann man dann direkt ablesen, welche Wörter in welchen Dokumenten vorkommen.

<br>

#### Count-based Vectors
Im Gegensatz zu den One-Hot-Vectors können hier die Vektorkomponenten verschiedene Werte >= 0 annehmen. Die Herangehensweise ist die selbe,
jedoch wird für jedes Wort nun nicht gespeichert, ob es vorkommt oder nicht, sondern wie oft. Somit lässt sich aus der Designmatrix später ablesen, welche Wörter **wie oft** in welchen Dokumenten vorkommen.

<br>

#### Context-based Word Vectors
Für bestimmte NLP Probleme lässt sich das Konzept auch insofern erweitern, dass man für jedes Wort dessen Kontext speichert.
Dabei wird eine bestimmte Fensterlänge n festgelegt, und für jedes Wort w(i) im Vokabular wird gespeichert, welches Wort w(j) in der 
n-Umgebung von Wort w(i) vorkommt. Dies ist wichtig, da sich herausgestellt hat, dass die Bedeutung eines Wortes maßgeblich dadurch
bestimmt wird, welche Wörter in seiner Umgebung / in seinem Kontext vorkommen. 
<br>
Dies lässt sich dann in einer Co-Occurence-Matrix darstellen, 
die die Form V x V besitzt (V = Vokabulargröße).

<br>

## Co-Occurence-Matrix
Eine Co-Occurence_Matrix für das Vokabular V = { all, that, glitters, is, not, gold, well, ends }, der Fenstergröße n = 1 und den zwei Dokumenten:
<br>
<br>
Document 1: "all that glitters is not gold"

Document 2: "all is well that ends well"

würde somit folgendermaßen aussehen:
<br>

|     *    | `<START>` | all | that | glitters | is   | not  | gold  | well | ends | `<END>` |
|----------|-------|-----|------|----------|------|------|-------|------|------|-----|
| `<START>`    | 0     | 2   | 0    | 0        | 0    | 0    | 0     | 0    | 0    | 0   |
| all      | 2     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| that     | 0     | 1   | 0    | 1        | 0    | 0    | 0     | 1    | 1    | 0   |
| glitters | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| is       | 0     | 1   | 0    | 1        | 0    | 1    | 0     | 1    | 0    | 0   |
| not      | 0     | 0   | 0    | 0        | 1    | 0    | 1     | 0    | 0    | 0   |
| gold     | 0     | 0   | 0    | 0        | 0    | 1    | 0     | 0    | 0    | 1   |
| well     | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 1    | 1   |
| ends     | 0     | 0   | 1    | 0        | 0    | 0    | 0     | 1    | 0    | 0   |
| `<END>`      | 0     | 0   | 0    | 0        | 0    | 0    | 1     | 1    | 0    | 0   |

<br>

#### Dimensionality Reduction / Word Embeddings
Da solche Matrizen in der Realität extrem groß werden und viele 0-Werte enthalten, was unter Anderem zu einem hohen Speicherverbrauch führt, verbessert man sie durch **Dimensionality Reduction**, was alle Dimensionen außer den **k wichtigsten** aus den Vektoren eliminiert und so aus den ursprünglichen Vektoren in der Matrix sogenannte **Word embeddings** formt.<br>
Diese stellen Information mit deutlich weniger und dafür informativeren Einträgen dar. Konkret lässt sich sagen, dass diese die (semantischen) Beziehungen zwischen den Wörtern trotz der geringeren Anzahl an Einträgen erhalten.

<br>

In [18]:
# Dimensionality Reduction with scikit-learn

M = np.array([
    [0,2,0,0,0,0,0,0,0,0],
    [2,0,1,0,1,0,0,0,0,0],
    [0,1,0,1,0,0,0,1,1,0],
    [0,0,1,0,1,0,0,0,0,0],
    [0,1,0,1,0,1,0,1,0,0],
    [0,0,0,0,1,0,1,0,0,0],
    [0,0,0,0,0,1,0,0,0,1],
    [0,0,1,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,1,0,0]
])

# Truncated SVD is for _small_ k top components
SVD = TruncatedSVD(n_components=2, n_iter=10)
M_reduced = SVD.fit_transform(M)

print("Original Matrix:\n {}\n".format(M))
print("Reduced Matrix:\n {}\n".format(M_reduced))


Original Matrix:
 [[0 2 0 0 0 0 0 0 0 0]
 [2 0 1 0 1 0 0 0 0 0]
 [0 1 0 1 0 0 0 1 1 0]
 [0 0 1 0 1 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 1]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 1 0 0]]

Reduced Matrix:
 [[ 1.35181032 -0.36285434]
 [ 0.38189729  2.29882445]
 [ 1.77174355 -0.30640105]
 [ 0.21740554  1.06081987]
 [ 1.8015794  -0.31544332]
 [ 0.16499552  0.63726979]
 [ 0.24724139 -0.05811929]
 [ 0.65538376  0.52269722]
 [ 0.60297374  0.09914715]]



## Wortähnlichkeit
Mit dem bisherigen Wissen lassen sich nun Ähnlichkeiten zwischen Wörtern berechnen. Dabei geht man oft korpusbasiert vor, d.h. man lädt einen geeigneten Korpus, stellt eine Co-Occurence-Matrix mit geeigneter Kontextgröße auf und erstellt Word embeddings durch SVD.<br>
Das allein liefert aber noch keine besonders guten Ergebnisse, u.a. weil besonders häufige und daher nicht informative Wörter die Ähnlichkeitsberechnung zu massiv beeinflussen.<br><br>
Es zeigt sich, dass wahrscheinlichkeitsbasierte Modelle mit Gewichtung nach Informativität (Word2Vec, GloVe) gute Performanz erzielen.

<br>

Eine mögliche Gewichtung ist die **Positive Pointwise Mutual Information (PPMI)**, welche misst, wie sehr das Zusammenauftreten von zwei Wörtern von der durch die Einzelwahrscheinlichkeiten der Wörter zu berechnenden, erwarteten Häufigkeit abweicht. Davon werden nur die positiven Korrelationen einbezogen, da diese am ausgeprägtesten sind.

<br>

![Pointwise Mutual Information](img/PMI.jpg)

<br>

Für die PPMI ergibt sich also die Formel max(0, PMI(w1,w2).<br>
Die Wahrscheinlichkeiten werden wieder anhand der relativen Häufigkeiten geschätzt:

<br>

![Probabilities for PMI](img/probabilities-pmi.jpg)

<br>

In [2]:
    # Load GloVe vectors (Note: this might take some minutes to load)
    glove_vecs = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(glove_vecs.vocab.keys()))
    
    print(glove_vecs.most_similar("oil"))
    # >>> 'petroleum', 'crude', 'gas', ....


Loaded vocab size 400000


---


# Spelling Correction
Man unterscheidet zwischen zwei Fehlerarten, die auftreten können:
<br>
- Non word errors: Fehler, die als Resultat kein richtiges Wort erzeugen.<br>
**Beispiel:** umbrella => ubrella
<br>
<br>
- Real word errors: Fehler, die als Resultat trotzdem ein richtiges Wort erzeugen.<br>
**Beispiel:** Three => There

<br>

Erstere sind leicht erkennbar: Jedes Token, was nicht im Dictionary / Vokabular vorkommt, ist ein Error. Dies erfordert ein möglichst großes Vokabular, um möglichst viele Spelling Errors erkennen zu können.
<br>
Um diese zu beheben, geht man wie folgt vor:
1. Stelle Kandidaten auf, also Wörter die möglichst ähnlich dem "Error-Word" sind.
2. Wähle z. B. anhand **Shortest Weighted Edit Distance** ein Wort aus.
<br>

Um **Real word errors** zu beheben, geht man ähnlich vor:
1. Stelle Kandidaten auf, die eine ähnliche Aussprache besitzen
2. Stelle Kandidaten auf, die ähnlich buchstabiert werden.
3. Inkludiere das "Error-Word" im Candidate set.
4. Wähle einen Kandidaten aus anhand z.B. Noisy Channel, Classifier, etc.

<br>

## Noisy Channel Model
Das Noisy Channel Model berechnet für ein gegebenes, falsch geschriebenes Wort das korrekte Wort mit der höchsten Wahrscheinlichkeit:
<br>

![Noisy Channel Probability](img/noisy-channel.jpg)
<br>
**w** = Korrektes Wort (aus dem Candidate set)
<br>
**x** = falsch geschriebenes Wort
<br>

<br>

#### Candidate generation
Die Kandidaten werden wieder anhand der kleinsten Edit-Distanz zum falsch geschriebenen Wort berechnet (meist Edit-Distanz = 1,2). Besser ist es, auch Wörter einzubeziehen, die die selbe Aussprache haben. Generell lässt sich sagen, dass etwa 80% aller Errors Edit-Distanz 1 besitzen.
<br>

Beispiel:
<br>
![Candidate generation example](img/candidates-example.jpg)

<br>

#### Error probability
Was ebenfalls benötigt wird sind die Wahrscheinlichkeiten des Error Models, d.h. für ein falsch geschriebenes Wort x und ein korrektes Wort w

<p style="text-align: center;">x = x1,x2,....,xm</p>
<p style="text-align: center;">w = w1,w2,....,wn</p>
<br>

möchten wir **P(x | w)**, also die "Wahrscheinlichkeit der Korrektur" ausrechnen.
<br>
Dafür benötigt man jeweils eine confusion matrix für Deletion, Insertion, Substitution und Transposition (Swapping) mit (im Fall Englisch) 26 Zeilen und 26 Spalten, welche jedem Buchstaben eine Zahl zuordnet, wie oft dieser fälschlicherweise durch einen anderen Buchstaben getippt wird.
<br>

Beispiel:
<br>
![Confusion matrix für Substitution](img/confusion-spelling-error.jpg)
<br>

Hat man diese Matrizen erstellt, lässt sich nun **P(x | w)** berechnen:
<br>
![Error model probabilities](img/error-probabilities.jpg)


In [12]:
letters = "abcdefghijklmnopqrstuvwxyz"
letter_to_idx = {letter: idx for idx, letter in enumerate(letters)}

array_sub = [
    [0,0,7,1,342,0,0,2,118,0,1,0,0,3,76,0,0,1,35,9,9,0,1,0,5,0],
    [0,0,9,9,2,2,3,1,0,0,0,5,11,5,0,10,0,0,2,1,0,0,8,0,0,0],
    [6,5,0,16,0,9,5,0,0,0,1,0,7,9,1,10,2,5,39,40,1,3,7,1,1,0],
    [1,10,13,0,12,0,5,5,0,0,2,3,7,3,0,1,0,43,30,22,0,0,4,0,2,0],
    [388,0,3,11,0,2,2,0,89,0,0,3,0,5,93,0,0,14,12,6,15,0,1,0,18,0],
    [0,15,0,3,1,0,5,2,0,0,0,3,4,1,0,0,0,6,4,12,0,0,2,0,0,0],
    [4,1,11,11,9,2,0,0,0,1,1,3,0,0,2,1,3,5,13,21,0,0,1,0,3,0],
    [1,8,0,3,0,0,0,0,0,0,2,0,12,14,2,3,0,3,1,11,0,0,2,0,0,0,],
    [103,0,0,0,146,0,1,0,0,0,0,6,0,0,49,0,0,0,2,1,47,0,2,1,15,0],
    [0,1,1,9,0,0,1,0,0,0,0,2,1,0,0,0,0,0,5,0,0,0,0,0,0,0],
    [1,2,8,4,1,1,2,5,0,0,0,0,5,0,2,0,0,0,6,0,0,0,4,0,0,3],
    [2,10,1,4,0,4,5,6,13,0,1,0,0,14,2,5,0,11,10,2,0,0,0,0,0,0],
    [1,3,7,8,0,2,0,6,0,0,4,4,0,180,0,6,0,0,9,15,13,3,2,2,3,0],
    [2,7,6,5,3,0,1,19,1,0,4,35,78,0,0,7,0,28,5,7,0,0,1,2,0,2],
    [91,1,1,3,116,0,0,0,25,0,2,0,0,0,0,14,0,2,4,14,39,0,0,0,18,0],
    [0,11,1,2,0,6,5,0,2,9,0,2,7,6,15,0,0,1,3,6,0,4,1,0,0,0],
    [0,0,1,0,0,0,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,14,0,30,12,2,2,8,2,0,5,8,4,20,1,14,0,0,12,22,4,0,0,1,0,0],
    [11,8,27,33,35,4,0,1,0,1,0,27,0,6,1,7,0,14,0,15,0,0,5,3,20,1],
    [3,4,9,42,7,5,19,5,0,1,0,14,9,5,5,6,0,11,37,0,0,2,19,0,7,6],
    [20,0,0,0,44,0,0,0,64,0,0,0,0,2,43,0,0,4,0,0,0,0,2,0,8,0],
    [0,0,7,0,0,3,0,0,0,0,0,1,0,0,1,0,0,0,8,3,0,0,0,0,0,0],
    [2,2,1,0,1,0,0,2,0,0,1,0,0,0,0,7,0,6,3,3,1,0,0,0,0,0], 
    [0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0],
    [0,0,2,0,15,0,1,7,15,0,0,0,2,0,6,1,0,7,36,8,5,0,0,1,0,0],
    [0,0,0,7,0,0,0,0,0,0,0,7,5,0,0,0,0,2,21,3,0,0,0,0,3,0]
]

matrix_sub = np.array(array_sub)
    
def get_matrix_value(matrix, letter_a, letter_b):
    idx_1 = letter_to_idx[letter_a]
    idx_2 = letter_to_idx[letter_b]
    return matrix[idx_1, idx_2]

# How often is m substituted by n?
print(get_matrix_value(matrix_sub, 'm', 'n'))


180
